In [41]:
import os
import sys
import requests
from bs4 import BeautifulSoup
from requests_html import HTMLSession

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36'}

class User:
    def __init__(self):
        print('press 1 for Sign Up')
        print()
        print('Already have an account?')
        print('press 2 for Log In')
        print()
        i = int(input('Enter your choise: '))
        if i == 1:
            User.SignUp(self)
        elif i == 2:
            User.LogIn(self)
        else:
            print('invalid choice')
            User.__init__(self)
        
    def SignUp(self):
        print()
        uname = input('Enter Username: ').strip()
        password = input('password: ').strip()
        
        os.chdir("C:\\Users\\luhar\\Anaconda-Jupyter-To Practice Programs\\Investment Tracker")
        with open('Users.txt', 'r') as f:
            users = []
            while True:
                data = f.readline()
                if data == '':
                    break
                else:
                    users.append(data.split('|')) 
            
            checker = False
            for i in range(len(users)):
                if users[i][0] == uname:
                    checker = True
                    
            if checker:
                print('Username Allready Taken!')
                User.SignUp(self)
            else:
                print('Creating New User...')
                balance = int(input('Add Money to Start Buying'))
                with open('Users.txt', 'a') as f:
                    f.write(uname+'|'+password+'|'+str(balance)+'|\n')
                os.chdir("C:\\Users\\luhar\\Anaconda-Jupyter-To Practice Programs\\Investment Tracker\\User_portfolios")
                with open(f'{uname}.txt', 'w')as f:
                    pass
                print('New User Created')
        
    def LogIn(self):
        print()
        uname = input('Enter your Username: ').strip()
        password = input('Enter your Password: ').strip()
        users = []
        User_Pass = {}
        balance = 0
        os.chdir("C:\\Users\\luhar\\Anaconda-Jupyter-To Practice Programs\\Investment Tracker")
        with open('Users.txt', 'r') as f:
            for line in f.readlines():
                users.append(line.split('|'))
        for i in range(len(users)):
            User_Pass[users[i][0]]=users[i][1]
            balance = users[i][2]
        if uname in User_Pass:
            if password == User_Pass[uname]:
                inside = Stock(uname, balance)
            else:
                print('Wrong Password')
        else:
            print('There is no user by this name!')
            print()
            print('1. Retry   2. Create new by Sign Up 3. Exit')
            print()
            choise = int(input('Enter Your Choise: '))
            if choise == 1:
                self.LogIn()
            elif choise == 2:
                self.SignUp()
            else:
                print('Exited')
        
        
class Stock:
    def __init__(self, uname, balance):
        self.uname = uname
        self.balance = balance
        self.name = ''
        self.company_prize = ''
        self.price_change = ''
        self.change = ''
        self.portfolio()
        
    def profitORloss(self):
        os.chdir('C:\\Users\\luhar\\Anaconda-Jupyter-To Practice Programs\\Investment Tracker\\User_portfolios')
        print()
        l = []
        l_short = []
        with open(f'{self.uname}.txt', 'r') as f:
            for line in f.readlines():
                l.append(line.split('|')[0])
                
        for i in l:
            string = i
            start_index = string.rfind('(') + 1
            end_index = string.rfind(')')
            
            snbraces = string[start_index:end_index]
            l_short.append(snbraces)
            
        l_detail = []
        with open(f'{self.uname}.txt', 'r') as f:
            for line in f.readlines():
                l_detail.append(line.split('|'))
                
        portfolio_value = 0
        current_value = 0
        
        for t, i in zip(l_short, l_detail):
            #finding current prize
            company = t
            url = f'https://finance.yahoo.com/quote/{company}?p={company}&.tsrc=fin-srch'
            page = requests.get(url, headers=headers)
            soup = BeautifulSoup(page.text, 'html.parser')
                
            name = soup.find('h1', {'class': 'D(ib) Fz(18px)'}).text.strip()
            company_prize = soup.find('fin-streamer', {'class': 'Fw(b) Fz(36px) Mb(-4px) D(ib)'}).text.strip()
            
            #finding counts we have
            count = 0
            
            
            if t in i[0]:
                count = int(i[4])
                portfolio_value += (float(i[1])*count)
            
            current_value += (float(company_prize)*count)
        
        print()
        print('Invested: ', portfolio_value)
        print('Current Valuation: ', current_value)
        print()
        
        p_OR_l = current_value - portfolio_value 
        
        if p_OR_l >= 0:
            print('Profit: ', p_OR_l)
        else:
            print('Loss: ',p_OR_l)
        self.portfolio()
            
            
    def portfolio(self):
        os.chdir('C:\\Users\\luhar\\Anaconda-Jupyter-To Practice Programs\\Investment Tracker\\User_portfolios')
        print()
        print('-'*30)
        print('|         PORTFOLIO          |')
        print('-'*30)
        with open(f'{self.uname}.txt', 'r') as f:
            for line in f.readlines():
                detail = ''
                ln = line.split('|')
                for i in ln:
                    detail += (i+'  ')
                print(detail)
        print('-'*30)
        print()
        print('Your Balance : ', self.balance)
        print()
        print('-'*30)
        print()
        print("Navigations")
        print("1. Buy New/Watch Prize  2. Sell  3. Your Valuation Details  4. Exit")
        print()
        choise = int(input("Enter Your Choise: "))
        if choise == 1:
            self.display()
        elif choise == 2:
            self.sell()
        elif choise == 3:
            self.profitORloss()
        elif choise == 4:
            print('Exited')
        else:
            print('Please Choose Valid Choise !')
            self.portfolio()
        
    def display(self):
        print()
        company = input('Company Name (by Symbole): ').upper().strip()
        url = f'https://finance.yahoo.com/quote/{company}?p={company}&.tsrc=fin-srch'
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.text, 'html.parser')
            
        self.name = soup.find('h1', {'class': 'D(ib) Fz(18px)'}).text.strip()
        self.company_prize = soup.find('fin-streamer', {'class': 'Fw(b) Fz(36px) Mb(-4px) D(ib)'}).text.strip()
        self.price_change = soup.find('div', {'class': 'D(ib) Mend(20px)'}).find_all('span')[0].text
        self.change = soup.find('div', {'class': 'D(ib) Mend(20px)'}).find_all('span')[1].text
        
        print('Loading :', url)
        print(self.name, ':', self.company_prize, self.price_change, self.change)
        print('-'*30)
        
        print('1. Search Another  2. Buy this 3. Back')
        b = int(input('Enter your choise: '))
        if b == 1:
            self.display()
        elif b == 2:
            self.buy()
        elif b == 3:
            print('Exited')
            self.portfolio()
            
            
    def buy(self):
        print()
        count = int(input('Enter Count: '))
        os.chdir('C:\\Users\\luhar\\Anaconda-Jupyter-To Practice Programs\\Investment Tracker\\User_portfolios')
        size = os.path.getsize(f'{self.uname}.txt')
        if size == 0:
            print ('Congratulations! You Are Buying Your FIRST Share')
        with open(f'{self.uname}.txt', 'a') as f:
            f.write(self.name+'|'+self.company_prize+'|'+self.price_change+'|'+self.change+'|'+str(count)+'|\n')
            
        os.chdir('C:\\Users\\luhar\\Anaconda-Jupyter-To Practice Programs\\Investment Tracker')
        l = []
        with open('Users.txt', 'r+') as f:
            for line in f.readlines():
                l.append(line.split('|'))
        for i in l:
            if i[0] == self.uname:
                i[2] = str(float(i[2]) - (float(self.company_prize)*count))
                
        with open('Users.txt', 'w') as f:
            for i in l:
                f.write('|'.join(i))
        print('Share Bought Successfully')
        self.portfolio()
        
    
    def sell(self):
        self.company_prize = ''
        print()
        print('-'*30)
        print()
        s_name = input('Enter Stock Name You Wish to Sell: ').upper().strip()
        count = int(input('Enter Count: '))
        t = s_name
        
        l = []
        cnt = 0
        with open(f'{self.uname}.txt', 'r') as f:
            for line in f.readlines():
                l.append(line.split('|'))
        
        sn = ''
        full_name = False
        for i in l:
            if t in i[0]:
                full_name = True
                sn = i[0]

        stock_name = sn
        
        if full_name == True:
            for i in l:
                if i[0] == sn:
                    cnt += int(i[4]) 
                    
            string = sn
            start_index = string.rfind('(') + 1
            end_index = string.rfind(')')
            
            snbraces = string[start_index:end_index]
            
            if cnt >= count:
                company = snbraces
                url = f'https://finance.yahoo.com/quote/{company}?p={company}&.tsrc=fin-srch'
                page = requests.get(url, headers=headers)
                soup = BeautifulSoup(page.text, 'html.parser')

                self.company_prize = soup.find('fin-streamer', {'class': 'Fw(b) Fz(36px) Mb(-4px) D(ib)'}).text.strip()

                os.chdir('C:\\Users\\luhar\\Anaconda-Jupyter-To Practice Programs\\Investment Tracker')
                ll = []
                with open('Users.txt', 'r+') as f:
                    for line in f.readlines():
                        ll.append(line.split('|'))
                for i in ll:
                    if i[0] == self.uname:
                        i[2] = str(float(i[2]) + (float(self.company_prize)*count))
                with open('Users.txt', 'w') as f:
                    for i in ll:
                        line = '|'.join(i)
                        f.write(line)

                for i in l:
                    if i[0] == sn:
                        if int(i[4]) <= count:
                            count -= int(i[4])
                            l.remove(i)
                        elif int(i[4]) > count:
                            i[4] = str(int(i[4]) - count)
                
                os.chdir('C:\\Users\\luhar\\Anaconda-Jupyter-To Practice Programs\\Investment Tracker\\User_portfolios')
                with open(f'{self.uname}.txt', 'w') as f:
                    for i in l:
                        line = '|'.join(i)
                        f.write(line)
                print('\n\n')      
                print('Share Sold Successfully')
                self.portfolio()
                            
            else:
                print('\n\n')
                print('You Have Insufficient Number of Share !')
                self.portfolio()
        else:
            print('\n\n')
            print("You Don't Have Share by This Name !")
            self.portfolio()
                 
user = User()

press 1 for Sign Up

Already have an account?
press 2 for Log In

Enter your choise: 2

Enter your Username: sujal
Enter your Password: 123

------------------------------
|         PORTFOLIO          |
------------------------------
Amazon.com, Inc. (AMZN)  100.61  +2.90  (+2.97%)  11  
  
The Boeing Company (BA)  204.70  -0.07  (-0.03%)  30  
  
Ford Motor Company (F)  11.72  +0.54  (+4.83%)  15  
  
Tesla, Inc. (TSLA)  197.58  +14.33  (+7.82%)  3  
  
Apple Inc. (AAPL)  159.28  +1.88  (+1.19%)  2  
  
Tesla, Inc. (TSLA)  197.58  +14.33  (+7.82%)  5  
  
------------------------------

Your Balance :  20690.139999999996

------------------------------

Navigations
1. Buy New/Watch Prize  2. Sell  3. Your Valuation Details  4. Exit

Enter Your Choise: 1

Company Name (by Symbole): sony
Loading : https://finance.yahoo.com/quote/SONY?p=SONY&.tsrc=fin-srch
Sony Group Corporation (SONY) : 89.37 +0.95 (+1.07%)
------------------------------
1. Search Another  2. Buy this 3. Back
Enter your